In [4]:
import pandas as pd
import numpy as np
import re

In [6]:
opentable_df = pd.read_csv('/Users/alyafattah/Desktop/recommendation-system-master/model/rest.csv')

In [7]:
opentable_df.head()

,Unnamed: 0,website_id,full_name,cuisine,price_range,reviews,rating,location,url,rest_id
0,0,1265,Obicà Mozzarella Bar - Poland Street,Italian,2,581.0,4.3,Oxford Circus,https://www.opentable.co.uk/obica-mozzarella-b...,2270
1,1,1496,Ham Yard Bar and Restaurant,Modern European,2,3697.0,4.6,Soho,https://www.opentable.co.uk/r/ham-yard-bar-and...,2259
2,2,1212,Hachi Japanese BBQ Restaurant,Japanese,2,270.0,4.6,Soho,https://www.opentable.co.uk/r/hachi-japanese-b...,2598
3,3,1241,El Norte,Spanish,4,299.0,4.5,Mayfair,https://www.opentable.co.uk/r/el-norte-london?...,2451
4,4,1441,Bocconcino,Italian,3,1525.0,4.0,Mayfair,https://www.opentable.co.uk/bocconcino?avt=eyJ...,2224


In [8]:
#Remove duplicates, if any 
opentable_df.drop_duplicates(subset = ['full_name'], inplace = True)

#Remove rows where there are no reviews
opentable_df = opentable_df[opentable_df['reviews'].notna()]

#Standarize caps
opentable_df['full_name'] = opentable_df['full_name'].str.lower()
opentable_df['full_name'] = opentable_df['full_name'].str.title()

In [11]:
#Remove rows with irrelevant cuisines
opentable_df = opentable_df[opentable_df.cuisine != 'Bar / Lounge / Bottle Service']
opentable_df = opentable_df[opentable_df.cuisine != 'Afternoon Tea']
opentable_df = opentable_df[opentable_df.cuisine != 'Breakfast']
opentable_df = opentable_df[opentable_df.cuisine != 'Pub']
opentable_df = opentable_df[opentable_df.cuisine != 'Dessert']
opentable_df = opentable_df[opentable_df.cuisine != 'Cocktail Bar']

In [12]:
opentable_df.cuisine.unique()

array(['Italian', 'Modern European', 'Japanese', 'Spanish', 'French',
       'Steak', 'Steakhouse', 'American', 'British',
       'Contemporary French', 'European', 'Chinese', 'Sushi',
       'Rotisserie Chicken', 'Middle Eastern', 'Korean', 'Seafood',
       'Russian', 'Thai', 'Indian', 'Greek', 'Tapas / Small Plates',
       'Contemporary British', 'Dim Sum', 'Chinese (Sichuan)',
       'Pizza Bar', 'Caribbean', 'Vietnamese', 'Turkish', 'Fish',
       'Lebanese', 'Asian', 'Californian', 'International',
       'Contemporary European', 'Latin American', 'Japanese (Yakiniku)',
       'Contemporary Indian', 'Mediterranean', 'Pizzeria', 'Vegan',
       'English', 'Mexican', 'Burgers', 'Global, International'],
      dtype=object)

In [13]:
cuisine_dict = {"Fish" : 'Seafood', "European": "Modern European", 
"Tapas / Small Plates": 'Spanish', "Contemporary British": "British", "Californian": "American",
"Burgers": "American", "Steakhouse": "Steak", 
"Dim Sum": "Chinese","Chinese (Sichuan) ": "Chinese", "Pizza Bar": "Italian", 
"Contemporary European": "Modern European","Lebanese": "Middle Eastern", 
"Turkish":"Middle Eastern", "Contemporary French": "French", "Japanese (Yakiniku)": "Japanese", 
"Vegan": "Vegetarian / Vegan", "English": "British", "Contemporary Indian": "Indian", 
"Global, International": "International", "Pizzeria": "Italian", "Sicilian": "Italian", 
"Argentinian": "Steak","Mexican": "Latin American", "Caribbean": "Latin American",  
"Sushi": "Japanese", "Continental":"International", "Rotisserie Chicken":"Others",
"Mediterranean":"Others"}

In [14]:
opentable_df =opentable_df.replace({"cuisine": cuisine_dict})

In [16]:
opentable_df.cuisine.unique()

array(['Italian', 'Modern European', 'Japanese', 'Spanish', 'French',
       'Steak', 'American', 'British', 'Chinese', 'Others',
       'Middle Eastern', 'Korean', 'Seafood', 'Russian', 'Thai', 'Indian',
       'Greek', 'Chinese (Sichuan)', 'Latin American', 'Vietnamese',
       'Asian', 'International', 'Vegetarian / Vegan'], dtype=object)

In [15]:
opentable_df

,Unnamed: 0,website_id,full_name,cuisine,price_range,reviews,rating,location,url,rest_id
0,0,1265,Obicà Mozzarella Bar - Poland Street,Italian,2,581.0,4.3,Oxford Circus,https://www.opentable.co.uk/obica-mozzarella-b...,2270
1,1,1496,Ham Yard Bar And Restaurant,Modern European,2,3697.0,4.6,Soho,https://www.opentable.co.uk/r/ham-yard-bar-and...,2259
2,2,1212,Hachi Japanese Bbq Restaurant,Japanese,2,270.0,4.6,Soho,https://www.opentable.co.uk/r/hachi-japanese-b...,2598
3,3,1241,El Norte,Spanish,4,299.0,4.5,Mayfair,https://www.opentable.co.uk/r/el-norte-london?...,2451
4,4,1441,Bocconcino,Italian,3,1525.0,4.0,Mayfair,https://www.opentable.co.uk/bocconcino?avt=eyJ...,2224
...,...,...,...,...,...,...,...,...,...,...
295,295,1399,Archipelago,International,4,396.0,4.6,Fitzrovia,https://www.opentable.co.uk/archipelago-london...,2479
296,296,1162,The Garden At Corinthia London,Modern European,2,181.0,4.6,Charing Cross,https://www.opentable.co.uk/r/the-garden-at-co...,2512
297,297,1310,Wild Honey Terrace At Sofitel,Others,3,17.0,4.6,Charing Cross,https://www.opentable.co.uk/r/wild-honey-terra...,2561
298,298,1188,Da Henrietta,Italian,3,369.0,4.2,Covent Garden,https://www.opentable.co.uk/r/da-henrietta-lon...,2451


In [19]:
opentable_df.to_csv("/Users/alyafattah/Desktop/alya_open_table.csv")